In [2]:
import pandas as pd

In [4]:
data_trial = pd.read_csv("../data/tsd_trial.csv")
print(data_trial.shape)
data_trial.head()

(690, 2)


,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."
3,"[87, 88, 89, 90, 91, 92]",Trump Claimed that Russia will never invade th...
4,[],As long as your willing to pay a lot more for ...


In [6]:
data_train = pd.read_csv("../data/tsd_train.csv")
print(data_train.shape)
data_train.head()

(7939, 2)


,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."


In [7]:
data = pd.concat([data_trial, data_train])
data.shape

(8629, 2)

Vemos que había solamente cinco textos repetidos. 

Se usará `tsd_train.csv` para entrenar y `tsd_trial` para validación

-------------

# Alfabeto

Vendría bien conocer que símbolos hay en los textos

In [16]:
conjuntos = data_train.text.agg(set)
alfabeto = set()

In [19]:
for index,item in conjuntos.iteritems():
    alfabeto |= item

In [20]:
alfabeto

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '}',
 '~',
 '\x7f',
 '\xa0',
 '«',
 '¬',
 '·',
 '»',
 'é',
 'ê',
 'ï',
 'ó',
 'ü',
 'ʻ',
 'ˈ',
 '͞',
 '\u2004',
 '\u200b',
 '–',
 '—',
 '―',
 '‘',
 '’',
 '“',
 '”',
 '•',
 '…',
 '™',
 '▀',
 '☠',
 '☭',
 '☹',
 '⚽',
 '⚾',
 '✭',
 '️',
 '🆘',
 '👎',
 '💀',
 '💥',
 '💨',
 '🔥',
 '😁',
 '😂',
 '😅',
 '😆',
 '😈',
 '😉',
 '😊',
 '😜',
 '😞',
 '😡',
 '😬',
 '😵',
 '🙄',
 '🤥'}

In [67]:
len(alfabeto)

150

Aparte del abecedario, tenemos algunos símbolos que se usan con relativa frecuencia en internet.

Sin embargo, también notamos que hay caracteres raros como las comillas dobles, guiones con sus variantes, etc. También hay que notar que hay emojis 

# Vocabulario

Para obtener el vocabulario, es necesario realizar un preprocesamiento a los textos, esto incluye:
* sustituir minúsculas
* quitar emojis
* ¿quitar caracteres raros?

In [21]:
import re

In [54]:
def preprocesar(texto):
    texto = texto.lower() #minúsculas
    texto = re.sub("[\W_]", " ", texto) #carácteres raros
    texto = re.sub("\s+", " ", texto) # colapsando espaciados
    return texto

In [55]:
s = data_train.text[0]
s

'Another violent and aggressive immigrant killing a innocent and intelligent US Citizen.... Sarcasm'

In [56]:
preprocesar(s)

'another violent and aggressive immigrant killing a innocent and intelligent us citizen sarcasm'

In [58]:
texto_preprocesado = data_train.text.apply(preprocesar)

In [64]:
texto_tokenizado = texto_preprocesado.apply(str.split)

vocabulario = set()
for index, lista in texto_tokenizado.iteritems():
    vocabulario |= set(lista)

In [66]:
len(vocabulario)

18992

# ¿Qué es tóxico?

Vamos a obtener el texto que se ha considerado como tóxico

In [86]:
def procesar_span(span):
    span = re.sub("\s","",span[1:-1])
    if span:
        lista = span.split(",")
        lista = [int(index) for index in lista]
    else:
        lista = []
    return lista

In [87]:
data_train["spans"] = data_train["spans"].apply(procesar_span)

In [120]:
def obtener_texto_toxico(row):
    toxico = []
    if row.spans: # Hay texto marcado como toxico
        diffs = np.diff(row.spans, prepend=0)
        toxico = []
        indices_inicios = np.argwhere(diffs != 1).squeeze()
        if indices_inicios: # mas de un cacho toxico
            i = j = 0
            for inicio in indices_inicios:
                j = inicio
                parte_toxica = row.text[i:j]
                toxico.append(parte_toxica)
                i = inicio
        else: # solamente un cacho
            i = row.spans[0]
            j = row.spans[-1]
            toxico.append(row.text[i:j+1])

    return toxico

In [124]:
obtener_texto_toxico(data_train.loc[3])

['knucklehead']

In [117]:
data_train.loc[0]

spans    [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...
text     Another violent and aggressive immigrant killi...
Name: 0, dtype: object

In [119]:
data_train.loc[0, "spans"]

[8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39]

In [116]:
obtener_texto_toxico(data_train.loc[0])

TypeError: iteration over a 0-d array

In [110]:
x = [1,2,3,4,5,8,9,10,11,12,16,17,18]
xd = np.diff(x, n=1, prepend=0)

In [112]:
np.argwhere(xd != 1).squeeze()

array([ 5, 10])

In [113]:
x[5]

8